In [ ]:


# Function to detect faces in an image
def detect_faces(image_path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    return img, faces

# Function to perform face matching and draw bounding boxes
def match_faces(reference_image, group_photo_folder, model_name="Facenet"):
    ref_embedding = DeepFace.represent(reference_image, model_name=model_name)[0]["embedding"]
    
    matched_images = []
    non_matched_images = []

    for img_name in os.listdir(group_photo_folder):
        img_path = os.path.join(group_photo_folder, img_name)
        img, faces = detect_faces(img_path)
        
        matched = False  # Flag to check if an image has a matching face

        for (x, y, w, h) in faces:
            face_crop = img[y:y+h, x:x+w]
            try:
                group_embedding = DeepFace.represent(face_crop, model_name=model_name)[0]["embedding"]
                similarity = np.dot(ref_embedding, group_embedding) / (np.linalg.norm(ref_embedding) * np.linalg.norm(group_embedding))

                if similarity > 0.6:  # Matching threshold
                    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 9)  # Draw bounding box
                    matched_images.append(img)  # Append the modified image
                    matched = True  # Mark image as matched
                    break  # Stop checking further faces if one matches
            except:
                continue

        if not matched:
            non_matched_images.append(img)  # If no match, add to non-matched list

    return matched_images, non_matched_images

# Paths
reference_image = r"sri.jpg"
group_photo_folder = "google_drive_photos"

# Perform face matching
matched_images, non_matched_images = match_faces(reference_image, group_photo_folder)

# Display matched images with bounding boxes
print("✅ Matched Images:")
for i, img in enumerate(matched_images):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for correct color display
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis("off")  # Hide axes
    plt.title(f"Matched Image {i+1}")
    plt.show()

# Display non-matched images
print("\n❌ Non-Matched Images:")
for i, img in enumerate(non_matched_images):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for correct color display
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis("off")  # Hide axes
    plt.title(f"Non-Matched Image {i+1}")
    plt.show()

print(f"✅ Total Matched Photos: {len(matched_images)}")
print(f"❌ Total Non-Matched Photos: {len(non_matched_images)}")
